# Imports

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow
tensorflow.random.set_seed(404)
from numpy.random import seed
seed(888)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import os
import numpy as np


from time import strftime
from PIL import Image

# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

In [4]:
TOTAL_INPUTS

784

# Get the Data

In [5]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 182 ms, sys: 1.74 ms, total: 184 ms
Wall time: 217 ms


In [6]:
y_train_all.shape

(60000,)

In [7]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [8]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 40.9 s, sys: 2.38 s, total: 43.3 s
Wall time: 1min 49s


In [9]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 6.91 s, sys: 177 ms, total: 7.09 s
Wall time: 14.6 s


# Explore

In [10]:
x_train_all.shape

(60000, 784)

In [11]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [12]:
y_train_all.shape

(60000,)

In [13]:
x_test.shape

(10000, 784)

In [14]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [15]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

#### Convert target values to one-hot encoding

In [16]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [17]:
y_train_all.shape

(60000, 10)

In [18]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [19]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [20]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [21]:
x_train.shape

(50000, 784)

In [22]:
x_val.shape

(10000, 784)

In [23]:
x_train_all.shape


(60000, 784)

# Setup Tensorflow Model

In [24]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture

#### Hyperparameters

In [25]:
nr_epochs = 20
learning_rate = 0.001
n_hidden1 = 512
n_hidden2 = 64

In [26]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out

In [27]:
## layer 1
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')
## Dropout Layer
layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

## Layer 2
layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')
## Output Layer
output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Loss, Optimisation & Metrics

#### Defining Loss Function

In [28]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [29]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [30]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Run Session

In [31]:
sess = tf.Session()

#### Initialise all the variables

In [32]:
init = tf.global_variables_initializer()
sess.run(init)

### Batching the Data

In [33]:
size_of_batch = 1000

In [34]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [35]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training Model

In [36]:
for epoch in range(nr_epochs):
    
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        sess.run(train_step, feed_dict=feed_dictionary)
        
    
    batch_accuracy = sess.run(fetches=accuracy, feed_dict=feed_dictionary)
        
    
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')

print('Done training!')

Epoch 0 	| Training Accuracy = 0.8629999756813049
Epoch 1 	| Training Accuracy = 0.8809999823570251
Epoch 2 	| Training Accuracy = 0.8960000276565552
Epoch 3 	| Training Accuracy = 0.9679999947547913
Epoch 4 	| Training Accuracy = 0.9679999947547913
Epoch 5 	| Training Accuracy = 0.9750000238418579
Epoch 6 	| Training Accuracy = 0.9750000238418579
Epoch 7 	| Training Accuracy = 0.9779999852180481
Epoch 8 	| Training Accuracy = 0.984000027179718
Epoch 9 	| Training Accuracy = 0.9819999933242798
Epoch 10 	| Training Accuracy = 0.9860000014305115
Epoch 11 	| Training Accuracy = 0.9869999885559082
Epoch 12 	| Training Accuracy = 0.9869999885559082
Epoch 13 	| Training Accuracy = 0.9860000014305115
Epoch 14 	| Training Accuracy = 0.9890000224113464
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9879999756813049
Epoch 17 	| Training Accuracy = 0.9879999756813049
Epoch 18 	| Training Accuracy = 0.9909999966621399
Epoch 19 	| Training Accuracy = 0.98799997

# Make a Prediction

In [37]:
img = Image.open('MNIST/test_img.png')
img

In [38]:
bw = img.convert('L')

In [39]:
img_array = np.invert(bw)

In [40]:
img_array.shape

(28, 28)

In [41]:
test_img = img_array.ravel()

In [42]:
test_img.shape

(784,)

In [43]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [44]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing and Evaluation

In [45]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.25%


# Close the Session

In [46]:
sess.close()